# Data Mining Versuch Verkehrsschilderkennung mit Neuronalen Netzen

* Autor: Prof. Dr. Johannes Maucher
* Datum: 01.06.2021

## Abgabe:

- **Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen und den entsprechenden Ausgaben.**
- **Das Notebook ist als .ipynb und als .html abzugeben.**
- **Klausurelevante Fragen sind Dokument "Fragenkatalog Datamining" zu finden.**
- Antworten auf Fragen im Notebook, Diskussionen und Beschreibung der Ergebnisse sind optional (aber empfohlen) und werden nicht bewertet.

* [Übersicht Data Mining Praktikum](https://maucher.pages.mi.hdm-stuttgart.de/ai/page/dm/)


# Einführung

In diesem Versuch soll ein Convolutional Neural Network (CNN) für die Erkennung von Verkehrschildern implementiert, trainiert, evaluiert und getestet werden. Als Eingabe erhält das neuronale Netz Bilder von deutschen Verkehrsschildern. Ausgabe ist der Typ des Verkehrsschilds. Für Training und Test sind die Verkehrsschildbilder schon in separate Verzeichnissen abgelegt. Neben den Bildern selbst, enthält das zu diesem Versuch gehörende Datenverzeichnis auch Dateien mit Metadaten, die z.B. Bildeigenschaften, Verkehrsschildbedeutungen und die zugehörigen Klassenlabel beschreiben. Die Daten können Sie von hier herunterladen: https://cloud.mi.hdm-stuttgart.de/s/2mTmkPejeP8s9NL

## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* Convolutional Neural Networks (CNNs)
* Implementierung Tiefer Neuronaler Netze mit Tensorflow und Keras: 
    - Definition der Netzarchitektur
    - Training
    - Evaluation und Test
    
* Einfache Methoden der Bildverarbeitung:
    - Augmentierung
    - Kontrastverstärkung

* Evaluation eines Klassifikators


## Vorbereitung

### Grundlagen Neuronale Netze

Machen Sie sich mit den [Grundlagen herkömmlicher Neuronaler Netze (KI Vorlesung)](https://lectures.mi.hdm-stuttgart.de/mi7ai/06NeuralNets.html) und den [Grundlagen Convolutional Neural Networks ((KI Vorlesung))](https://lectures.mi.hdm-stuttgart.de/mi7ai/06ConvolutionNeuralNetworks.html) vertraut (**user**: *mi7ai*, **pw**: *ailecture*).


### Implementierung Neuronaler Netze mit Tensorflow und Keras

Machen Sie sich mit der Implementierung von Neuronalen Netzen mit Tensorflow und Keras vertraut. Z.B. mit den [Tensorflow Quickstart Tutorials](https://www.tensorflow.org/tutorials/quickstart/beginner)

# Durchführung

In [86]:
import skimage as ski
from skimage.transform import resize
from skimage.exposure import equalize_adapthist
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report

In [87]:
tf.config.run_functions_eagerly(False)

## Vorbereitende Untersuchungen
1. Importieren Sie ein Bild aus dem Verzeichnis `Train` mit der [scikit-image.io](https://scikit-image.org/docs/stable/api/skimage.io.html)-Methode `imread()` und zeigen Sie dieses mit der Methode `imshow()` an. Geben Sie die Größe des Bildes aus (Attribut `.shape`). 

In [ ]:
selected_img = ski.io.imread('traffic-sign-recognition\\data\\Train\\0\\00000_00000_00000.png')
plt.imshow(selected_img)

In [ ]:
selected_img.shape

2. Verändern Sie die Größe des Bildes mit der [scikit-image.transform](https://scikit-image.org/docs/stable/api/skimage.transform.html)-Methode `resize()` auf eine Größe von $32x32x3$. Die Verzerrung des Seitenverhältnisses kann dabei ignoriert werden. Diese Methode führt auch eine Normalisierung der Pixelwerte von [0,255] auf [0,1] durch.

In [ ]:
selected_img = resize(selected_img, (32, 32, 3))
selected_img.shape

3. Führen Sie mit der [scikit-image.exposure](https://scikit-image.org/docs/stable/api/skimage.exposure.html)-Methode `equalize_adapthist()` eine Kontrastverstärkung des Bildes durch. Zeigen Sie das vergrößerte und kontrastangereicherte Bild an. **Anmerkung:** Das kontrastverstärkte Bild sieht zwar unschöner aus, auf der Basis kontrastverstärkter Bilder läßt sich aber im allgemeinen die Objekterkennung verbessern.

In [ ]:
selected_img = equalize_adapthist(selected_img)
ski.io.imshow(selected_img)

In [ ]:
selected_img.shape

4. Importieren Sie die Datei `Train.csv` und machen Sie sich mit deren Inhalt vertraut. Die Datei `Test.csv` ist gleich strukturiert, bezieht sich aber auf die Bilder im Verzeichnis `Test`. Wieviele Zeilen enthalten die Dateien?

In [ ]:
train_data = pd.read_csv('traffic-sign-recognition\\data\\Train.csv')
train_data.head()

In [ ]:
print(f'Die Train.csv enthält {train_data.shape[0]} Zeilen')

In [ ]:
test_data = pd.read_csv('traffic-sign-recognition\\data\\Test.csv')
test_data.head()

In [ ]:
print(f'Die Test.csv enthält {test_data.shape[0]} Zeilen')

5. Importieren Sie die Datei `signnames.csv` und machen Sie sich mit deren Inhalt vertraut. 

In [ ]:
signnames = pd.read_csv('traffic-sign-recognition\\data\\signnames.csv')
signnames.head()

## Funktion für den Import aller Trainings- bzw. Testbilder
Schreiben Sie eine Funktion mit folgenden Eigenschaften:
* Der Funktion wird der Name der Datei übergeben, in welcher die Metadaten stehen, also entweder `Train.csv` oder `Test.csv`.
* Rückgabewerte der Funktion sind 
    * ein 4-dimensionales numpy-array, das alle Bilder des jeweiligen Verzeichnisses (Training oder Test) enthält.
    * ein 1-dimensionales numpy-array, das die Klassenlabel aller Bilder enthält.
* Die Bilder müssen alle auf eine Größe von $32x32x3$ skaliert werden (wie in der Vorbereitungsaufgabe).
* Für alle Bilder ist eine Kontrastverstärkung durchzuführen (wie in der Vorbereitungsaufgabe).

**Tipps für die Implementierung dieser Funktion:**

Iterieren Sie mit einer for-Schleife über alle Zeilen des metadaten-Files. Pro Iteration kann dann 
* der vollständige Verzeichnis- und Filenamen ausgelesen werden,
* das entsprechende Bild mit `imread()` eingelesen werden,
* das Bild auf die vorgegebene Größe angepasst werden,
* der Kontrast des Bildes verstärkt werden.

**Wichtig:** In den von der Funktion zurückgegebenen Arrays, dürfen die Bilder nicht wie in der ursprünglichen Reihenfolge im Dateiverzeichnis enthalten sein. Um sicherzustellen, dass beim Training jedes Minibatch möglichst viele verschiedene Klassen enthält, muss die Reihenfolge geshuffelt werden. Am einfachsten ist es, wenn gleich die Zeilen des Metadatenfiles geshuffelt werden.

In [12]:
def import_file(file_name):
    file_path = f'traffic-sign-recognition\\data\\{file_name}.csv'
    data = pd.read_csv(file_path)
    data = data.sample(frac=1.0, random_state=42).reset_index(drop=True)

    imgs_number = data.shape[0]
    images = np.zeros((imgs_number, 32, 32, 3), dtype=np.float32)
    labels = np.zeros(imgs_number, dtype=np.float32)
    
    for index, row in data.iterrows():
        img = ski.io.imread(f'traffic-sign-recognition\\data\\{row["Path"]}')
        img = resize(img, (32, 32, 3))
        img = equalize_adapthist(img)

        images[index] = img
        labels[index] = row['ClassId']
    
    return images, labels

## Laden und Vorverarbeiten der Trainings- und Testdaten
1. Laden Sie mit der in der vorigen Teilaufgabe implementierten Funktion alle Trainingsbilder (`trainX`), Trainingslabel (`trainY`), Testbilder (`testX`) und Testlabel (`testY`)

In [13]:
trainX, trainY = import_file('Train')

In [ ]:
trainX.shape, trainY.shape

In [15]:
testX, testY = import_file('Test')

In [ ]:
testX.shape, testY.shape

2. Bestimmen Sie die Häufigkeitsverteilung der Klassen in den Trainings- und Testdaten. Visualisieren Sie diese.

In [ ]:
labels_train, l_count_train = np.unique(trainY, return_counts=True)
labels_test, l_count_test = np.unique(testY, return_counts=True)

df = pd.DataFrame({'label': labels_train, 'count_train': l_count_train, 'count_test': l_count_test})

# Create the stacked bar plot
plt.figure(figsize=(15, 6))
plt.bar(df['label'], df['count_train'], label='count_train', color='green')
plt.bar(df['label'], df['count_test'], bottom=df['count_train'], label='count_test', color='red')

# Customize the plot
plt.xlabel('Label')
plt.ylabel('Counts')
plt.title('Distribution of labels in the data')
plt.legend()
plt.xticks(df['label'])
plt.tight_layout()

# Show the plot
plt.show()

3. Alle Labels, sowohl der Trainings- als auch der Testdaten müssen One-Hot-encodiert werden.

In [18]:
lb = LabelBinarizer()

In [19]:
lb.fit(trainY)
trainY_ohe = lb.transform(trainY)

In [20]:
lb.fit(testY)
testY_ohe = lb.transform(testY)

## Definition der CNN Architektur
Schreiben Sie eine Funktion `generateCNN(width, height, depth, classes)` die eine Keras CNN-Architektur zurück gibt. Für die Definition der Architektur werden dieser Funktion die Parameter:

* `width`: Breite der Bilder
* `height`: Höhe der Bilder
* `depth`: Anzahl der Kanäle pro Bild
* `classes`: Anzahl der unterschiedlichen Klassen

übergeben. Die in der Funktion zu implementierende Architektur ist im folgenden Bild dargestellt. In der Spalte *Output shape* bezeichnen die zweite und dritte Zahl die Breite und die Höhe der einzelnen Kanäle (*Bilder*), der letzte Parameter bezeichnet die Anzahl der Kanäle (Parameter *filters* in der Konfiguration).
In der Übersichtstabelle ist die Filtergröße nicht aufgeführt. Empfohlen sind folgende Größen:
* für alle Pooling Layer: *pool_size=(2,2)*.
* für den ersten Conv2D-Layer: *kernel_size=(5,5)*.
* für alle weiteren Conv2D-Layer: *kernel_size=(3,3)*

**Anmerkung:** Der in der Tabelle mit *flatten_5* bezeichnete Layer ist nicht notwendig und erzeugt in bestimmten Keras-Versionen eine Fehlermeldung. Der Layer sollte nicht in die Architektur mit aufgenommen werden.


<img src="https://maucher.home.hdm-stuttgart.de/Pics/cnnTrafficSign.png" style="width:400px" align="middle">

In [88]:
def generateCNN(width, height, depth, classes):

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5), input_shape=(width, height, depth), padding='same'),
        tf.keras.layers.Activation(activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same'),
        tf.keras.layers.Activation(activation='relu'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same'),
        tf.keras.layers.Activation(activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same'),
        tf.keras.layers.Activation(activation='relu'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same'),
        tf.keras.layers.Activation(activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=128),
        tf.keras.layers.Activation(activation='relu'),
        
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(rate=0.2),

        tf.keras.layers.Dense(units=128),
        tf.keras.layers.Activation(activation='relu'),

        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(units=classes),
        tf.keras.layers.Activation(activation='softmax')
        ])

    return model

## Training des CNN
Rufen Sie die im vorigen Abschnitt implementierte Funktion `generateCNN()` auf und weisen Sie die von der Funktion zurückgegebene Architektur der Variablen `model` zu. Durch Aufruf der Funktion `model.summary()` erhalten Sie eine Übersicht des erzeugten Netzes.

Für das Training soll der `Adam`-Algorithmus aus dem Modul `tensorflow.keras.optimizers` benutzt werden. `Adam` implementiert ein *Stochastic Gradient Descent*-Lernverfahren, welches die Lernraten für die Gewichte individuell und dynamisch anpasst.

In den folgenden zwei Codezellen, werden die Trainingsparameter konfiguriert:

In [89]:
model = generateCNN(32, 32, 3, 43)

c:\Users\danna\anaconda3\envs\DMaPR\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [90]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 32, 32, 8)      │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_64 (Activation)      │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 32, 32, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 16, 16, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_65 (Activation)      │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 16, 16, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 16, 16, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_66 (Activation)      │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 16, 16, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 8, 8, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_67 (Activation)      │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 8, 8, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 8, 8, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_68 (Activation)      │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_69 (Activation)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 107,019 (418.04 KB)

 Trainable params: 106,363 (415.48 KB)

 Non-trainable params: 656 (2.56 KB)

In [91]:
NUM_EPOCHS = 15 # Number of training epochs 
INIT_LR = 1e-3 # Initial Learning Rate for ADAM training
BS = 64 # Size of minibatches

In [92]:
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR) # rename lr to learning_rate
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

Für das Training sollen nicht nur die vorhandenen Trainingsbilder eingesetzt werden, sondern zusätzlich Bilder die Augmentierungen der Trainingsbilder sind. Augmentierte Bilder können mit dem `ImageDataGenerator` des Moduls `tensorflow.keras.preprocessing.image` erzeugt werden. Der Code für die Erzeugung des in diesem Projekt eingesetzten Objekts ist unten gegeben. 

**Aufgabe:** Erklären Sie was in dieser Codezelle definiert wird.

In [93]:
# construct the image generator for data augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

1. ``rotation_range``: Das Bild wird zufällig um 10 Grad gedreht
2. ``zoom_range``: Das Bild wird zufällig um bis zu 15% vergrößert oder verkleinert
3. ``width_shift_range``: Das Bild wird um bis zu 10% in seiner Breite verschoben
4. ``height_shift_range``: Das Bild wird um bis zu 10% in seiner Höhe verschoben
5. ``shear_range``: Das Bild wird mit maximal 15 Grad verzerrt
6. ``horizontal_flip``: Das Bild wird nicht horizontal geflippt
7. ``vertical_flip``: Das Bild wird nicht vertikal geflippt
8. ``fill_mode``: Auffüllung der leeren Bereich nach Transformation = die nächsten Pixel

Das Training wird mit folgender Codezelle ausgeführt.

**Aufgabe:** Erklären Sie die Argumente der Funktion `fit()`. 

Für die Ausführung der Zelle muss das Dictionary `classWeight` angelegt sein. Dieses enthält für jede Klasse den Klassenindex als key und den relativen Anteil dieser Klasse in den Trainingsbildern als Value. Wenn z.B. 30% aller Trainingsdaten zur Klasse 0 gehören, dann wäre der Value zum Key 0 der Wert 0.3.

1. ``aug`` ist der zuvor definierte ``ImageDataGenrator`` für Datenaugmentation. Der ``flow``-Befehl stellt sicher, dass die Daten in Batches geladen werden und gleichzeitig die Datenaugmentation angewedent wird. ``trainX`` sind die Trainingsdaten und ``trainY`` die zugehörigen Labels. ``BS`` ist die oben definierte Batchgröße.
2. Die Testdaten und Testlabels werden in ``validation_data`` verwendet, um die Leistung des Modells auf den Testdaten zu bewerten.
3. Bei ``steps_per_epoch`` wird die Anzahl der Schritte (Batches) pro Epoche definiert. Dabei ist ``trainX.shape[0]`` die Anzahl der Trainingsbeispiele und ``BS`` die Batchgröße.
4. `epochs` gibt mit ``NUM_EPOCHS`` die Anzahl der Epochen an, also wie of das Modell die gesamten Trainigsdaten durchlaufen soll.
5. `class_weight` stellt mit `classWeight` ein Dictionarie dar, welcher die Gewichte für jede Klasse aufweist.
6. `verbose` ist zuständig für die Informationsanzeige. Mit `1` wird der Fortschritt des Trainings angezeigt, mit der Epochennummer, Verlustfunktion und der Metriken.

In [94]:
labels_train, l_count_train = np.unique(trainY, return_counts=True)

classWeight_df = pd.DataFrame({'count_train': l_count_train/sum(l_count_train)})

classWeight = classWeight_df.to_dict()['count_train']

In [95]:
# compile the model and train the network
print("[INFO] training network...")
H = model.fit(
    aug.flow(trainX, trainY_ohe, batch_size=BS),
    validation_data=(testX, testY_ohe),
    steps_per_epoch=trainX.shape[0] // BS,
    epochs=NUM_EPOCHS,
    class_weight=classWeight,
    verbose=1)

[INFO] training network...
Epoch 1/15


c:\Users\danna\anaconda3\envs\DMaPR\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


612/612 ━━━━━━━━━━━━━━━━━━━━ 103s 158ms/step - accuracy: 0.3282 - loss: 0.0814 - val_accuracy: 0.6944 - val_loss: 1.0071
Epoch 2/15
  1/612 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.7031 - loss: 0.0265

c:\Users\danna\anaconda3\envs\DMaPR\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


612/612 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7031 - loss: 0.0265 - val_accuracy: 0.6960 - val_loss: 1.0066
Epoch 3/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 70s 115ms/step - accuracy: 0.7011 - loss: 0.0221 - val_accuracy: 0.7874 - val_loss: 0.7025
Epoch 4/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7656 - loss: 0.0143 - val_accuracy: 0.7922 - val_loss: 0.6812
Epoch 5/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 71s 116ms/step - accuracy: 0.7997 - loss: 0.0130 - val_accuracy: 0.8345 - val_loss: 0.5147
Epoch 6/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8438 - loss: 0.0119 - val_accuracy: 0.8342 - val_loss: 0.5138
Epoch 7/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 62s 101ms/step - accuracy: 0.8554 - loss: 0.0091 - val_accuracy: 0.8562 - val_loss: 0.4734
Epoch 8/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8594 - loss: 0.0075 - val_accuracy: 0.8560 - val_loss: 0.4710
Epoch 9/15
612/612 ━━━━━━━━━━━━━━━━━━━━ 63s 104ms/step - accuracy: 0.8821 - loss: 0.0073 - val_accuracy: 0

Visualisieren Sie die Entwicklung der *Accuracy* über dem Fortschritt der Trainingsepochen. Plotten Sie dabei die entsprechenden Kurven der Accuracy auf den Trainings- und auf den Testdaten in einen Graphen.

In [ ]:
plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Evaluation des gelernten Modells

Wenden Sie das gelernte CNN an, um für alle Bilder des Testdatensatzes die Art des Verkehrsschildes zu bestimmen. Evaluieren Sie die Qualität des CNN indem Sie einen `classification_report()` aus dem Modul `sklearn.metrics` erzeugen. 

In [ ]:
predictions = model.predict(testX)
predictions_labels = [np.argmax(predictions[index]) for index, i in enumerate(predictions)]

In [ ]:
print(classification_report(testY, predictions_labels))

1. Welche Metriken werden im Report angezeigt? Beschreiben Sie diese kurz?

Classification Report
- Ein classification Report enthält die folgenden Metriken für jede Klasse: Precision, Recall, F1-Scpre und Support

Intepretation:

- Precision:
    - Niedrige Precision in einer Klasse weist auf viele False Positives hin
    - Zu weniger Trainingdaten für eine Klasse
    - Feature ist nicht aussagekräftig genug für diese Klasse
- Recall:
    - Ein niedriger Recall zeigt, dass der Klassifikation viele tatsächlichen Instanzen der Klasse nicht erkennt
    - Zu weniger Trainingdaten für eine Klasse
    - Feature ist nicht aussagekräftig genug für diese Klasse
- F1-Score:
    - Diese Metrik ist wichtig, wenn Precision und Recall für eie Klasse stark variieren
- Accuracy: 
    - Gibt den Anteil der korrekt klassifizierten Datenpunkten an allen Datenpunkten an
    - 100% = Alle Datenpunkte wurden richtig erkannt
    - Funktioniert gut, wenn die Daten gleich verteilt sind
- Macro avg: 
    - Summe der einzelnen Metriken geteilt durch die Anzahl der Klassen (Arithmetisches Mittel)
- Weighted avg: 
    - Summe der einzelnen Metriken multipliziert mit dem jeweiligen Support der Klassen, geteilt durch die gesamt Anzahl der Testdaten.$$ \sum_{k = 0}^{n} x_k * (\frac{support}{totalSupport}) $$

n = Anzahl der Klassen


$x_k$ = Wert der Metrik der k-ten Klasse

2. Diskutieren Sie die Klassifikationsgenauigkeit des CNN anhand des Reports.

Die Genauigkeit des Modells liegt bei ca. 90% (&plusmn; 5%, je nach Trainingsablauf), damit dist das Modell recht zuverlässig.
Precision und Recall sind bei weighted Average nah aneinander, was bedeuted dass das Modell gut angepasst ist. Der weighted Average ist aussagekräftiger als der Macro Average, da er die Verteilung der Klassen in den Testdaten berücksichtigt. In der echten Welt kommen Schilder unterschiedlich oft vor, weshalb die Berücksichtigung der Heufigkeit der Schilder relevant ist.

Zeigen Sie 5 Bilder an, die nicht korrekt klassifiziert wurden. Läßt sich die Fehlklassifikation erklären?

In [33]:
false_pred = []

for img, pred, y in zip(testX, predictions_labels, testY):
    if pred != y:
        false_pred.append((signnames[signnames['ClassId'] == pred].iloc[0]['SignName'], 
                           img, 
                           signnames[signnames['ClassId'] == y].iloc[0]['SignName']))

In [ ]:
print(false_pred[0][0])
print(false_pred[0][2])
plt.imshow(false_pred[0][1])

In [ ]:
print(false_pred[1][0])
print(false_pred[1][2])
plt.imshow(false_pred[1][1])

In [ ]:
print(false_pred[2][0])
print(false_pred[2][2])
plt.imshow(false_pred[2][1])

In [ ]:
print(false_pred[3][0])
print(false_pred[3][2])
plt.imshow(false_pred[3][1])

In [ ]:
print(false_pred[4][0])
print(false_pred[4][2])
plt.imshow(false_pred[4][1])

- Bei manchen lässt es sich nicht erklären, da für das menschliche Auge ein deutlicher Unterschied besteht (In Form und Farbe, Schneefloke verwechselt mit Fahrradfahrer).
- Bei manchen anderen liegt es offentsichtlich an der schlechten Qualität (Bsp. Geschwindigketsbegrenzung 50 und 80 / 90).
- Bei manchen verschwimmt das Straßenschild mit dem Hintergrund (Farbe und Form, schlecht oder zu stark beleuchtet).

## Hypothese: Die Erhöhung der Epochen beim Traiining des CNNs, steigert die Accuracy des Models, da es mehr Epochen hat um die verfügabren Merkmale zu lernen.


Epochgen: 30

In [ ]:
tf.executing_eagerly()

In [ ]:
tf.__version__

In [64]:
model_H = generateCNN(32, 32, 3, 43)

In [65]:
NUM_EPOCHS_H = 30 # Number of training epochs 

In [66]:
model_H.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# compile the model and train the network
print("[INFO] training network...")
H_H = model_H.fit(
    aug.flow(trainX, trainY_ohe, batch_size=BS),
    validation_data=(testX, testY_ohe),
    steps_per_epoch=trainX.shape[0] // BS,
    epochs=NUM_EPOCHS_H,
    class_weight=classWeight,
    verbose=1)

In [ ]:
plt.plot(H_H.history['accuracy'])
plt.plot(H_H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predictions = model_H.predict(testX)
predictions_labels = [np.argmax(predictions[index]) for index, i in enumerate(predictions)]

In [ ]:
print(classification_report(testY, predictions_labels))